# import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline
import re

from collections import Counter

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn import preprocessing

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

pd.set_option('display.max_colwidth', -1)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [ ]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [ ]:
sample_submission.head(5)

In [ ]:
sample_submission.info()

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [ ]:
data['URL_TA']

In [ ]:
data.sample(5)

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

# 2. Обработка признаков
## 2.1 Обработка ресторан ID

In [ ]:
def edit_rest_id(df_input):    
    # full edit restaurant_id 
    
    df = df_input.copy()
    
    # создания признака один ли ресторан или это сеть
    df['Restaurant_id'] = df['Restaurant_id'].apply(lambda x: int(x[3:]))
    chain_restaurant = df.groupby('Restaurant_id')['City'].count().rename('one_or_more')
    df = pd.merge(df, chain_restaurant, on='Restaurant_id')
    
    return df

In [ ]:
edit_rest_id(data).sample(2)

## 2.2 Обработка City

In [ ]:
data['City'].unique()

Дополнительным признаком можно добавить население. Городов не очень много, поэтому буду просто гуглить. Мне понравился сайт https://datacommons.org/
Буду считать что население не сильно изменилось за пару лет, поэтому не буду обращать внимание на год, за который представлены данные населения

In [ ]:
# население для каждого города из датасета
population = {
    'Paris': 2160928,
    'Helsinki': 631695,
    'Edinburgh': 482005,
    'London': 8982256,
    'Bratislava': 424428,
    'Lisbon': 504718,
    'Budapest': 1756056,
    'Stockholm': 789024,
    'Rome': 2873494,
    'Milan': 1351562,
    'Munich': 1471508,
    'Hamburg': 1841179,
    'Prague': 1308632,
    'Vienna': 1897491,
    'Dublin': 544107,
    'Barcelona': 1620343,
    'Brussels': 174383,
    'Madrid': 3223334,
    'Oslo': 634293,
    'Amsterdam': 869709,
    'Berlin': 3644826,
    'Lyon': 513275,
    'Athens': 664046,
    'Warsaw': 1764615,
    'Oporto': 214349,
    'Krakow': 766683,
    'Copenhagen': 602481,
    'Luxembourg': 114303,
    'Zurich': 402762,
    'Geneva': 198979,
    'Ljubljana': 279631              
}

In [ ]:
# принадлежность города к стране
country = {
    'Paris': 'France',
    'Hamburg': 'Germany',
    'Rome': 'Italy',
    'London': 'UK',
    'Milan': 'Italy',
    'Madrid': 'Spain',
    'Oslo': 'Norway',
    'Stockholm': 'Sweden',
    'Krakow': 'Poland',
    'Lyon': 'Paris',
    'Lisbon': 'Portugal',
    'Edinburgh': 'UK',
    'Vienna': 'Austria',
    'Warsaw': 'Poland',
    'Amsterdam': 'Netherlands',
    'Budapest': 'Hungary',
    'Helsinki': 'Finland',
    'Zurich': 'Switzerland',
    'Luxembourg': 'Luxembourg',
    'Berlin': 'Germany',
    'Prague': 'Czechia',
    'Munich': 'Germany',
    'Bratislava': 'Slovakia',
    'Brussels': 'Belgium',
    'Ljubljana': 'Slovenia',
    'Copenhagen': 'Denmark',
    'Oporto': 'Portugal',
    'Barcelona': 'Spain',
    'Geneva': 'Switzerland',
    'Athens': 'Greece',
    'Dublin': 'Ireland'
}

In [ ]:
# вышло так, что городов не столиц оказалось меньше, поэтому что меньше писать составлю из них список
not_Capital = ['Barcelona', 'Milan', 'Hamburg', 'Munich','Lyon', 'Zurich', 'Oporto', 'Geneva', 'Krakow']

In [ ]:
def create_dummy(df_input, series, n=10):
    # one hot encoding for n popular elements
    df = df_input.copy()    
    # list of n top elements
    top_elements = df[series].value_counts().index[:n]
    
    df[series] = df[series].apply(lambda x: x if x in top_elements else 'other')
    
    # create dummy for elements in top_elements
    dummy = pd.get_dummies(df[series])
    df = pd.concat([df, dummy], axis=1)
    
    return df

In [ ]:
def edit_city(df_input):
    # full edit city and about city
    
    df = df_input.copy()
    scaler = StandardScaler()
    
    # население каждого города
    df['population'] = df['City'].map(population)

    # Определение страны для каждого города из датасета
    df['country'] = df['City'].map(country)

    # сколько ресторанов в каждом городе
    restaurant_in_city = df.groupby('City')['Reviews'].count().rename('restaurant_in_city')
    df = pd.merge(df, restaurant_in_city, on='City')

    # количество ресторанов на каждого человека
    df['restaurant_per_person'] = df['restaurant_in_city'] / df['population']
    df['restaurant_per_person'] = scaler.fit_transform(df[['restaurant_per_person']])

    # столица или не столица
    df['capital'] = df['City'].apply(lambda x: 0 if x in not_Capital else 1 )

    return df    

In [ ]:
edit_city(data).sample(2)

Возможно надо подумать над признаком, туристический ли это город или нет. Или сколько денег приносит в бюджет города, но пока что не знаю как это реализовать.

Были создананы следующие признаки:
- население каждого города
- принадлежность города к стране
- количество ресторанов для каждого города
- из этого можно получить количество ресторанов на каждого человека
- является ли город столицей 
- dummy переменные (видел в других нотбуках label endocting, но не знаю зачем его применять к этому признаку)

## 2.3 Кухни

In [ ]:
def only_top_cuisins(string):
#     возвращает только популярные кухни
    output = []

    string_to_list = re.findall(r"'(\b.*?\b)'", string)
    # проверка для каждой входит ли в топ
    for cuisin in string_to_list:
        if cuisin in top_cuisins:
            output.append(cuisin)
    if not output:
        return ['Other']
    return output

In [ ]:
def edit_cuisine(input_df, n=10):
    # edit all about cuisine style
    
    df = input_df.copy()
    global top_cuisins
    
    # создаю столбец, с отметками о том, где были пропуски
    df['Cuisine_style_ISNA'] = df['Cuisine Style'].isna().astype(int)

    # заполнение пропусков
    df['Cuisine Style'] = df['Cuisine Style'].fillna("['Other']")
    
    # топ кухонь по популярности
    # перебор список все списков видов кухни
    all_cuisins = df['Cuisine Style'].str.findall(r"'(\b.*?\b)'")
    cuisins = []
    for list_cuisins in all_cuisins:
        for style in list_cuisins:
            cuisins.append(style)
    # создание списка n популярных кухонь
    set_cuisins = set(cuisins)        
    cuisins = dict(Counter(cuisins))
    cuisins = {keys: values for keys, values in sorted(cuisins.items(), reverse=True, key=lambda x: x[1])}
    top_cuisins = list(cuisins.keys())[:n]
    
    # возвращаю только топ n кухонь, остальные other
    df['Cuisine Style'] = df['Cuisine Style'].apply(lambda x: only_top_cuisins(x))

    # разнообразие кухонь
    df['len_cuisine'] = df['Cuisine Style'].apply(len) 
    
    # созднаие dummy переменных для видов кухни
    for cuisin in top_cuisins:
        df[cuisin] = df['Cuisine Style'].apply(lambda x: 1 if cuisin in x else 0)   
    
    return df

In [ ]:
edit_cuisine(data, n=5).sample(2)

В признаке достаточно много пропусков
- создал столбец с пометками о пропусках
- пропуски заменил значением other
- создал признак разнообразия кухонь
- создал функцию для создания n популярных dummy переменных

## 2.4 Price Range

In [ ]:
# хочу посмотреть самый популярный диапазон цен для каждого города
cities = data['City'].unique()
# отображение самых популярных диапазонов цен для каждого города
for city in cities:
    print(data[data['City']==city]['Price Range'].value_counts().index[0], city)

In [ ]:
def edit_price_range(input_df):
    # edit all about price range
    
    df = input_df.copy()
    
    # перед обработкой признака, создаю колонку с отображениями пропуска
    df['Price_Range_isNAN'] = df['Price Range'].isnull().astype(int)

    # выглядит так, что пропуски можно заполнить через fillna самым популярным значением
    df['Price Range'] = df['Price Range'].fillna('$$ - $$$')

    # замена значений типа объект на циферки 
    replace_price_range = {'$': 0, '$$ - $$$': 1, '$$$$':2}
    df['Price Range'] = df['Price Range'].map(replace_price_range)
    
    return df

In [ ]:
edit_price_range(data).sample(2)

Никаких новых знаний получить не удалось. В колонке было очень много пропусков. Отменил в датасете этот факт. Заполнил пропуски самым популярным значением. Произвел Label Encoding

## 2. 5 number of reviews

In [ ]:
def edit_number_of_reviews(input_df):
    # all about number_of_reviews
    
    df = input_df.copy()
    scaler = StandardScaler()
    
    # отмечу строки с пропусками, для них создам специальный столбец
    df['Number_of_Reviews_isNAN'] = df['Number of Reviews'].isnull().astype(int)

    # заполнение средними по каждому городу, но надо до каонца разобраться с синтаксисом конструкции трансформ
    df['Number of Reviews'] = df.groupby("City")['Number of Reviews'].transform(lambda x: x.fillna(x.mean()))
    
    
    
    
        # средний ранкинг для каждого города
    mean_per_city = df.groupby('City')['Number of Reviews'].mean()
    df['mean_Number_of_Reviews_per_city'] = df['City'].apply(lambda x: mean_per_city[x])

    #  масмимальный ранкинг для каждого города
    max_per_city = df.groupby('City')['Number of Reviews'].max()
    df['max_Number_of_Reviews_per_city'] = df['City'].apply(lambda x: max_per_city[x])

    # стандартизация
    df['stand_Number_of_Reviews'] = (df['Ranking'] - df['mean_Number_of_Reviews_per_city']) / df['max_Number_of_Reviews_per_city']
    
    
    
    
    

    # количество отзывов на каждого человека в городе
    try:
        df['reviews_per_each_person'] = df['Number of Reviews'] / df['population']
    except:
        pass

    # среднее количество отзывов по городам
    reviews_per_city = df.groupby(by='City')['Number of Reviews'].mean()
    df['reviews_per_city'] = df['City'].apply(lambda x: reviews_per_city[x])
    
    try:
        df['reviews_per_each_person'] = scaler.fit_transform(df[['reviews_per_each_person']])
    except:
        pass
    
    return df

In [ ]:
edit_city(edit_number_of_reviews(data)).sample(2)

Данные достаточно чистые, но там где были пропуски все равно был создан столбец с пометкой об этом. 
- пропуски заполнены средними значениями для каждого города, есть еще вариант заполнить пропуски нулями, но вывести новый столбец для средних значения для каждого города
- создан столбец с количеством отзывов для каждого человека

## 2.6 Ranking

In [ ]:
# распределение параметра ранкинг. Много значений, которые даже не дотягивают до 2500 места в своем городе
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

In [ ]:
# самые ресторанные города, если можно так сказать
data['City'].value_counts(ascending=True).plot(kind='barh')

In [ ]:
data['Ranking'][data['City'] == 'London'].hist(bins=100)
# не знаю почему это распределение называют нормальным. Для Лондона следующее распределение. Посмотрим на распределение других городов

In [ ]:
for x in (data['City'].value_counts())[0:10].index:
    data['Ranking'][data['City'] == x].hist(bins=100)
plt.show()

# Здесь видно что форма распределения повторяется, но видно смещение, это связано с размерами города

In [ ]:
def edit_ranking(df_input):
    # all about edit
    df = df_input.copy()    
    
    # приведем значения к виду от -1 до 1

    # средний ранкинг для каждого города
    mean_per_city = df.groupby('City')['Ranking'].mean()
    df['mean_ranking_per_city'] = df['City'].apply(lambda x: mean_per_city[x])

    #  масмимальный ранкинг для каждого города
    max_per_city = df.groupby('City')['Ranking'].max()
    df['max_ranking_per_city'] = df['City'].apply(lambda x: max_per_city[x])

    # стандартизация
    df['stand_ranking'] = (df['Ranking'] - df['mean_ranking_per_city']) / df['max_ranking_per_city']
    
    return df

In [ ]:
edit_ranking(data).sample(2)

In [ ]:
# посмотрим обновленое распределение для тех же городов
test_data = edit_ranking(data)
for x in (test_data['City'].value_counts())[0:10].index:
    test_data['stand_ranking'][test_data['City'] == x].hist(bins=100)
plt.show()

Признак без пропусков. Распределение одинаковое для каждого города. Можно заметить что дейсвительно смещеные было вызвано только размером города

## 2.7 обработка признака Reviews

In [ ]:
def days_between_dates(dates):

#     это функция высчитывает количество дней между отзывами
        
    if len(dates) == 0:
        return 1800
    if len(dates) == 1:
        return 3600
    dt_list = []
    for date in dates:
        dt = pd.to_datetime(date)
        dt_list.append(dt)
    return int((max(dt_list) - min(dt_list)).days)

In [ ]:
def edit_reviews(df_input):
    # all about edit reviews
    
    df = df_input.copy()
    

    # пропуски найденные с помощью isna() можно заполнить пустыми списками
    df['Reviews'] = df['Reviews'].fillna('[[], []]')

    # создам признак отображающий есть ли пропуск в данном признаке
    df['reviews_is_NAN'] = (df['Reviews'] == '[[], []]').astype(int)

    # признак состоит из двух последних отзывов и даты, в которой этот отзыв оставлен 
    # для начала можно попробовать извлечь значения дат и попробовать извлечь новые знания из этого
    df['date_of_reviews'] = df['Reviews'].str.findall('\d+/\d+/\d+')

    # сразу можно проверить, действительно ли содержится два отзыва
    df['len_date_of_reviews'] = df['date_of_reviews'].apply(len)
        
    # видно что некоторые пользователи записывали дату своего посещения
    # этот можно можно немного отредактировать
    df['date_of_reviews'] = df['date_of_reviews'].apply(lambda x: x[1:] if len(x) > 2 else x)

    # соответственно надо испрвить колонку len_date_of_reviews, т.к. значений = 3 больше нет
    df['len_date_of_reviews'] = df['date_of_reviews'].apply(len)
    
    # создание нового признака. Количество дней между последними отзывами
    df['days_between'] = df['date_of_reviews'].apply(days_between_dates)

    
    return df

In [ ]:
edit_reviews(data).head(2)

Пробуски в признаке встречаются. Так же есть признаки, где данные заполнены наполовину. Например 1 отзыв и соответственно одна дата.
- заполнены пропуски и создан признак с отметкой об этом
- создан признак с количеством отзывов для конкретного рестора
- создан признак перерыв между двумя датами. 

В дальнешей можно попробовать найти оттенок отзыва. При переходе на случайную ссылку, представленной в данном датасе я заметил что оценка которая стоит в датасете не совпадает с оценкой в ресторане, поэтому не уверен что необходимо парсить данные со страницы ресторанов. Можно попробовать в дальнейшем создать новый признак такой как в какое время суток чаще всего посещают каждое заведение, но опять таки. Я не знаю насколько информация из 2020 года будет актуально для 2017

## 2.8. URL_TA

In [ ]:
data['URL_TA'].value_counts()

In [ ]:
data.drop(['URL_TA'], axis=1, inplace=True)

Я думал что таким образом можнжо будет вычислить сетевые рестораны, но увы. Не знаю как может пригодиться этот признак, поэтому в дальнейшем я просто его удалю

## 2.9 ID_TA

In [ ]:
data['ID_TA'].value_counts()

In [ ]:
data['ID_TA'].apply(lambda x: x[1:])

In [ ]:
# data.drop(['ID_TA'], axis=1, inplace=True)

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [ ]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
        
    df = df_input.copy()
    
    ############################## Restaurant_id   ###################################    
    df = edit_rest_id(df)
    
    ############################## City   ###################################
    df = edit_city(df)
    df = create_dummy(df, 'City', n=31)
    df = create_dummy(df, 'country', n=15)
    
    ############################## Cuisine  ###################################
    df = edit_cuisine(df, n=30)
        
    ############################## Price Range  ###################################
    df = edit_price_range(df)
    
    ############################## Number of Reviews  ###################################
    df = edit_number_of_reviews(df)
    
    
    ############################## Ranking  ###################################        
    df = edit_ranking(df)
    
    ############################## Reviews  ###################################    
    df = edit_reviews(df)    
    
    
    df['ID_TA'] = df['ID_TA'].apply(lambda x: int(x[1:]))
    
    
    return df

#### Запускаем и проверяем что получилось

In [ ]:
df_preproc = preproc_data(data)
df_preproc.sample(2)

In [ ]:
# список переменных для корреляции
list_for_corr = ['Restaurant_id', 'Ranking', 'Price Range', 'Number of Reviews',
       'sample', 'Rating', 'one_or_more', 'population', 'restaurant_in_city',
       'restaurant_per_person', 'capital', 'Cuisine_style_ISNA', 'len_cuisine',
       'Price_Range_isNAN', 'Number_of_Reviews_isNAN',
       'mean_Number_of_Reviews_per_city', 'max_Number_of_Reviews_per_city',
       'stand_Number_of_Reviews', 'reviews_per_each_person',
       'reviews_per_city', 'mean_ranking_per_city', 'max_ranking_per_city',
       'stand_ranking', 'reviews_is_NAN', 'len_date_of_reviews',
       'days_between']

In [ ]:
plt.rcParams['figure.figsize'] = (20,8)
ax = sns.heatmap(df_preproc[list_for_corr].corr(), annot=True, fmt='.2g')
i, k = ax.get_ylim()
ax.set_ylim(i+0.5, k-0.5)

In [ ]:
drop_list = ['mean_ranking_per_city', 'max_ranking_per_city', 'population', 'mean_Number_of_Reviews_per_city']

df_preproc.drop(drop_list, axis=1, inplace=True)

In [ ]:
object_columns = df_preproc.select_dtypes(include='object').columns
df_preproc.drop(object_columns, axis = 1, inplace=True)

In [ ]:
df_preproc.info()

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

In [ ]:
test_data

In [ ]:
test_data = test_data.sort_values(by=['Restaurant_id'])

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)
y_pred = np.round(y_pred*2) / 2

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
test_data

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.sort_values(by=['Restaurant_id'])
test_data = test_data.drop(['Rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)
predict_submission = np.round(predict_submission*2) / 2

In [ ]:
predict_submission

In [ ]:
sample_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('new_submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
